In [18]:
import io
import fitz
import re
from PIL import Image, ImageDraw, ImageFont, ImageColor
import operator
import pandas as pd 
import numpy as np
from datetime import datetime
import math
from tqdm import tqdm

# Ploting funcs

In [19]:
def plot_genus_blocks(page_df, draw, color = '#6c899e', w = 3):
    try:
        genus_list = page_df['draw_genus'].unique()
    except:
        #print("no GENUS found")
        return 

    for g in genus_list:
        temp_df = page_df[(page_df['draw_genus'] == g)]
        g_x0 = temp_df['x0'].min()
        g_y0 = temp_df['y0'].min()
        g_x1 = temp_df['x1'].max()
        g_y1 = temp_df['y1'].max()

        draw.rectangle((g_x0, g_y0, g_x1, g_y1), fill=None, outline=ImageColor.getrgb(color), width = w)
        
def plot_epithet_blocks(page_df, draw, color = '#660066', w = 3):
    try:
        epithet_list = page_df['draw_epithet'].unique()
    except:
        print("no EPITHET found")
        return 
    
    for e in epithet_list:
        temp_df = page_df[(page_df['draw_epithet'] == e)]
        e_x0 = temp_df['x0'].min()
        e_y0 = temp_df['y0'].min()
        e_x1 = temp_df['x1'].max()
        e_y1 = temp_df['y1'].max()

        draw.rectangle((e_x0, e_y0, e_x1, e_y1), fill=None, outline=ImageColor.getrgb(color), width = w)

def plot_author_blocks(page_df, draw, color = '#a3a3a3', w = 1):
    try:
        author_list = page_df['draw_author'].unique()
    except:
        print("no AUTHOR found")
        return 

    for a in author_list:
        temp_df = page_df[(page_df['draw_author'] == a)]
        e_x0 = temp_df['x0'].min()
        e_y0 = temp_df['y0'].min()
        e_x1 = temp_df['x1'].max()
        e_y1 = temp_df['y1'].max()

        draw.rectangle((e_x0, e_y0, e_x1, e_y1), fill=None, outline=ImageColor.getrgb(color), width = w)

def plot_infra_blocks(page_df, draw, color = '#ff6289', w = 1):
    try:
        infra_list = page_df['draw_infra'].unique()
    except:
        print("no INFRA Spp. found")
        return 

    for infra_spp in infra_list:
        temp_df = page_df[(page_df['draw_infra'] == infra_spp)]
        e_x0 = temp_df['x0'].min()
        e_y0 = temp_df['y0'].min()
        e_x1 = temp_df['x1'].max()
        e_y1 = temp_df['y1'].max()

        draw.rectangle((e_x0, e_y0, e_x1, e_y1), fill=None, outline=ImageColor.getrgb(color), width = w)

def plot_valid_words(page_df, draw, color = '#660044', w = 2):
    blocks = page_df['block_no'].unique()
    """for b in blocks:
        lines = page_df[page_df['block_no'] == b]['line_no'].unique()
        for l in lines:
            cond = (page_df['line_no'] == l) & (page_df['block_no'] == b)
            words = page_df[cond]['word_no'].unique()
            page_df = page_df.copy()
            for w in words:
                x0 = page_df[(cond) & (page_df['word_no'] == w)]['x0'].item()
                y0 = page_df[(cond) & (page_df['word_no'] == w)]['y0'].item()
                x1 = page_df[(cond) & (page_df['word_no'] == w)]['x1'].item()
                y1 = page_df[(cond) & (page_df['word_no'] == w)]['y1'].item()
                draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb(color), width = w)
    """
    for index, row in page_df.iterrows():
        x0, y0, x1, y1 = row['x0'], row['y0'], row['x1'], row['y1'] 
        draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb(color), width = w)

# Import vol2 Index

In [20]:
pdf_dir = "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 2.pdf"
doc = fitz.open(pdf_dir)
pages = [doc[i] for i in range(doc.page_count)]
index = list(range(703, 725))

TARGET_DPI = 300
mat = fitz.Matrix(TARGET_DPI/ 72, TARGET_DPI/ 72)

indent_groups = []
indent_err = 15

In [21]:
"""R - Z pages"""
"""pdf_dir = "../input/vol2_r-z_final.pdf"
doc = fitz.open(pdf_dir)
pages = [doc[i] for i in range(doc.page_count)]
index = list(range(doc.page_count))

TARGET_DPI = 300
mat = fitz.Matrix(TARGET_DPI/ 72, TARGET_DPI/ 72)

indent_groups = []
indent_err = 15""";

# GET page_df

In [22]:
def get_page_df(page_num):

    #getting lines from block dicts
    """***START BLOCK SECTION***"""
    #words_df = pd.DataFrame(pages[page_num].get_text_words(), columns =['in_x0', 'in_y0', 'in_x1', 'in_y1', 'word', 'block_no', 'line_no', 'word_no'])
    #words_df['b_l_tuple'] = tuple(zip(words_df['block_no'], words_df['line_no']))
    #using get_text to extract 'size', 'flags', 'font'
    #NOTE: not sure if this is the best way to go about this ...
    content_blocks_df = pd.DataFrame(pages[page_num].get_text("dict")['blocks'])
    line_dicts = content_blocks_df[~content_blocks_df['lines'].isnull()].explode('lines', ignore_index=False)
    line_dicts

    #block_no is number
    #type is image vs text
    #NEW GOAL : for each exploded item --> get sub indexing for that region and use it as the 
    #                                       line_no
    #                                       word_no

    line_dicts.set_index(['number',line_dicts.groupby('number').cumcount()]).rename_axis(['block_no','line_no']).tail(10)
    line_dicts[["_", "line_no"]]=['number',line_dicts.groupby('number').cumcount()]
    line_dicts['idx'] = np.arange(line_dicts.shape[0])

    #df = df.explode('foo')
    #line_dicts.rename(columns={"number": "block_no"}) -- oh wasn't in place that's the problem lol
    """***END BLOCK SECTION***"""

    #from lines to single words per row
    """***START WORD SECTION***"""
    #THE MAIN ONE NOW
    pd.DataFrame(list(line_dicts['lines']))
    #content_df = pd.DataFrame(list((pd.DataFrame(list(line_dicts['lines']))['spans'].explode('spans'))))
    span_of_lines = pd.DataFrame(list(line_dicts['lines']))
    content_df_lines = pd.concat([line_dicts[["number", "type", "line_no"]].reset_index(), span_of_lines], axis=1)
    content_df = content_df_lines.explode('spans')

    """
    content_df = content_df[content_df['text'] != ' ']
    content_df['text'] = content_df['text']
    content_df['text'] = content_df['text'].apply(lambda x : list(x.split()))
    content_df = content_df.explode('text')
    split_bbox_df = pd.DataFrame(content_df['bbox'].tolist(), columns=['x0', 'y0', 'x1', 'y1'])
    pretty_content_df = pd.concat([content_df.reset_index(), split_bbox_df], axis=1)
    line_dicts.explode('lines')


    #https://stackoverflow.com/questions/38231591/split-explode-a-column-of-dictionaries-into-separate-columns-with-pandas
    """
    #pd.DataFrame(list((pd.DataFrame(list(line_dicts['lines']))['spans'].explode('spans'))))
    pd.DataFrame(list((pd.DataFrame(list(line_dicts['lines']))['spans'].explode('spans'))))
    (pd.DataFrame(list(line_dicts['lines']))['spans'].explode('spans'))
    """
    df = pd.DataFrame(list(line_dicts['lines']))['spans']
    df['span_no'] = np.arange(line_dicts['lines'].shape[0])
    pd.DataFrame(list(df.explode('spans')))""";
    pd.DataFrame(list((pd.DataFrame(list(line_dicts['lines']))['spans'].explode('spans')))).head(20)
    df = pd.DataFrame(list(line_dicts['lines'])).explode('spans')

    #df.set_index([df.index,line_dicts.groupby(df.index).cumcount()]).rename_axis(['line_no','span_no']).tail(10)
    df['idx_val'] = df.index
    df[["_", "span_no"]]=['idx_val',df.groupby('idx_val').cumcount()]
    #df = df.explode('foo')
    df.rename(columns={"number": "line_no"})
    #df[df['span_no'] > 0]
    df = pd.DataFrame(list(df['spans']), index= df['idx_val'])
    df['idx'] = df.index
    df[["_", "span_no"]]=['idx',df.groupby('idx').cumcount()]
    content_df = df 
    content_df = content_df[content_df['text'] != ' ']
    content_df['text'] = content_df['text']
    content_df['text'] = content_df['text'].apply(lambda x : list(x.split()))
    content_df = content_df.explode('text')
    split_bbox_df = pd.DataFrame(content_df['bbox'].tolist(), columns=['x0', 'y0', 'x1', 'y1'])
    pretty_content_df = pd.concat([content_df.reset_index(), split_bbox_df], axis=1)
    """***END WORD SECTION***"""

    #merging the lines df and the words df and pruning the extra content
    mergedRes = pd.merge(line_dicts, pretty_content_df, on ='idx')
    pruned = mergedRes[['number', 'line_no', 'idx', 'text', 'flags', 'font','x0', 'y0', 'x1', 'y1']]

    #creating tuples of block_no, line_no -- used to retrive word number
    pruned['b_l_tuple'] = tuple(zip(pruned['number'], pruned['line_no']))
    #getting word_no
    pruned[["_", "word_no"]]=['b_l_tuple',pruned.groupby('b_l_tuple').cumcount()]

    #changing number to block_no
    pruned.rename(columns = {'number':'block_no', 'x0':'in_x0', 'y0':'in_y0', 'x1':'in_x1', 'y1':'in_y1', 'text':'word'}, inplace = True)
    return pruned[['block_no', 'line_no', 'word_no', 'word', 'flags', 'font', 'in_x0', 'in_y0', 'in_x1', 'in_y1', 'b_l_tuple']]


# regex based boolean functions

In [23]:
def valid(word):
    """
    valid words are words that are:
    - at least 2 characters
        - unless it's x (symbol for hybrid)
    """
    return (not bool(re.search(r"[0-9]+[,.]?", word))) and \
            (word != 'NOUVELLE' and word != 'FLORE') and \
            (word != 'INDEX' and word != 'SPECIERUM') and \
            (len(word) > 1 or \
                word == 'x' or word == 'X' or word == '×' or word == r'\u00D7') and \
            ''.join(e for e in word if e.isalpha()).isalpha()
    
def is_genus(word):
    """
    A word in the index might be a genus if it satisfies the following properties:
    - letters: french alphabet + at most one hyphen (which is not first or last letter)
        - first letter upper case
        - all but first lowecase 
    in regex: ^[A-ZÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ]{1}[a-zàâäèéêëîïôœùûüÿç]*[-]?[a-zàâäèéêëîïôœùûüÿç]+$ #ignoring strict beggining and end cause of noise
        * based on the current expression it'd also be at least 2 letters long
    """
    regex = r"[A-ZÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ\u00D7]{1}[a-zàâäèéêëîïôœùûüÿç]*[-]?[a-zàâäèéêëîïôœùûüÿç]+"
    return re.search(regex, word)
    

def is_epithet(word):
    """
    A word in the index might be an epithet if it satisfies the following properties:
    - letters: french alphabet + at most one hyphen (which is not first or last letter)
        - all letters lowecase 
    in regex: ^[a-zàâäèéêëîïôœùûüÿç]+[-]?[a-zàâäèéêëîïôœùûüÿç]+$ #ignoring strict beggining and end cause of noise 
        * based on the current expression it'd also be at least 2 letters long
    """
    regex = r"[a-zàâäèéêëîïôœùûüÿç\u00D7]+[-]?[a-zàâäèéêëîïôœùûüÿç]+"
    return re.search(regex, word)
    
def is_hybrid(word):
    regex = r"^(([Xx\u00D7])|([Xx\u00D7]\.))$"
    return re.search(regex, word)

def is_infra(word):
    regex = r"^(var\.)|(subsp\.)"
    return re.search(regex, word)

# pre-processing func

In [24]:
def preprocessing(page_num, indent_err = 30):
    
    #initiate dataframe
    #using get_text_words to extract 'block_no', 'line_no', 'word_no'
    """
    words_df = pd.DataFrame(pages[page_num].get_text_words(), columns =['in_x0', 'in_y0', 'in_x1', 'in_y1', 'word', 'block_no', 'line_no', 'word_no'])
    words_df['b_l_tuple'] = tuple(zip(words_df['block_no'], words_df['line_no']))
    #using get_text to extract 'size', 'flags', 'font'
    #NOTE: not sure if this is the best way to go about this ...
    content_blocks_df = pd.DataFrame(pages[page_num].get_text("dict")['blocks'])
    line_dicts = content_blocks_df[~content_blocks_df['lines'].isnull()].explode('lines', ignore_index=False)

    content_df = pd.DataFrame(list((pd.DataFrame(list(line_dicts['lines']))['spans'].explode('spans'))))
    content_df = content_df[content_df['text'] != ' ']
    content_df['text'] = content_df['text']
    content_df['text'] = content_df['text'].apply(lambda x : list(x.split()))
    content_df = content_df.explode('text')
    split_bbox_df = pd.DataFrame(content_df['bbox'].tolist(), columns=['x0', 'y0', 'x1', 'y1'])
    pretty_content_df = pd.concat([content_df.reset_index(), split_bbox_df], axis=1)

    #have to take this step beforem merging the words don't match even when they should -- matching based on bbox won't help with this either :( 
    pretty_content_df = pretty_content_df[pretty_content_df["text"].apply(valid)].reset_index()
    words_df = words_df[words_df["word"].apply(valid)].reset_index()

    joined_df = pd.concat([pretty_content_df, words_df], axis=1)
    #the rows of each row should correspond. This following assertion assures that this is the case
    #TODO: might be worthwhile to match according to bbox coordinates (x0, y0, x1, y1) and in_x0, in_y0, in_x1, in_y1
    try:
        assert joined_df[joined_df['text'] != joined_df['word']].empty
    except:
        print("failed on page", page_num)

    page_df = joined_df[['in_x0', 'in_y0', 'in_x1', 'in_y1', 'word', 'text','block_no', 'line_no', 'word_no', 'flags', 'font', 'size', 'color', 'b_l_tuple']]
    """

    page_df = get_page_df(page_num)
    #initiate all columns that will be added
    page_df['page_num'] = np.array([page_num]*page_df.shape[0])
    page_df['genus'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_genus'] = np.array([np.NaN]*page_df.shape[0])
    page_df['epithet'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_epithet'] = np.array([np.NaN]*page_df.shape[0])
    page_df['author'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_author'] = np.array([np.NaN]*page_df.shape[0])
    page_df['infra'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_infra'] = np.array([np.NaN]*page_df.shape[0])
    page_df['taxon rank'] = np.array([np.NaN]*page_df.shape[0])
    page_df['error_check'] = np.array([np.NaN]*page_df.shape[0])
    
    #remove italics
    #italics_b_l = page_df[page_df['flags'] != 6]['b_l_tuple']
    #page_df = page_df[page_df['b_l_tuple'].apply(lambda x : x in italics_b_l.unique())]
    #italics_b_l = page_df[page_df['flags'] != 6]['b_l_tuple'].unique()
    #page_df = page_df[page_df['b_l_tuple'].isin(italics_b_l)]
    italics_b_l = page_df[page_df['flags'] == 6]['b_l_tuple'].unique()
    page_df = page_df.drop(page_df[page_df['b_l_tuple'].isin(italics_b_l)].index.tolist())
    
    #updating coordinates to represent target DPI
    page_df['x0'], page_df['y0'], page_df['x1'], page_df['y1'] = page_df['in_x0']*TARGET_DPI/ 72, page_df['in_y0']*TARGET_DPI/ 72, page_df['in_x1']*TARGET_DPI/ 72, page_df['in_y1']*TARGET_DPI/ 72
    #get x corner coordinates 
    x_min = page_df['x0'].min()
    x_max = page_df['x1'].max()

    y_max = page_df['y1'].max()

    #Remove the extra flore - 18 at page 545
    if page_num == index[4]:
        page_df = page_df[~((page_df["word"] == 'Flore') & (page_df['y1'] == y_max))]
    #invalid words dataframe -- for error checking
    pruned_words_df = page_df[~page_df["word"].apply(valid)].reset_index()
    page_df = page_df[page_df["word"].apply(valid)].reset_index()
    #prune out invalid words (based on function valid)
    #page_df = page_df[page_df["word"].apply(valid)].reset_index()
    
    indent_groups = []
    blocks = page_df['block_no'].unique()
    for b in blocks:
        lines = page_df[page_df['block_no'] == b]['line_no'].unique()
        for l in lines:
            #reset word_no values (useful for cases where word that was originally at 0th index was pruned out)
            cond = (page_df['line_no'] == l) & (page_df['block_no'] == b)
            num_words = len(page_df[cond]['word_no'])
            page_df.loc[cond, 'word_no'] = np.arange(num_words).astype(int) #this is slowww
            #set column number (0 or 1)
            x_0 = page_df[cond]['x0'].min()
            #THIS DOESN'T WORK AAAA -- issue was with line no thing
            if not np.isnan(x_0):
                page_df.loc[cond, 'col_no'] = np.array([int(x_0 > ((x_min + x_max) / 2))]*num_words).astype(int)

                #initiate indent groups -- only first word should get an indent_group value 
                new_group = True
                for g_i in range(len(indent_groups)):
                    g = indent_groups[g_i]
                    g_arr = np.array(g)
                    if x_0 <= np.mean(g_arr) + indent_err and x_0 >= np.mean(g_arr) - indent_err:
                        g.append(x_0)
                        new_group = False
                        page_df.loc[cond, 'indent_group'] = np.array([g_i]*num_words).astype(int)
                if new_group:
                    indent_groups.append([x_0])
                    g_i = len(indent_groups) - 1
                    page_df.loc[cond, 'indent_group'] = np.array([g_i]*num_words).astype(int)
            
    #print("indent groups:", indent_groups)
    #return updated page_df, pruned_words_df, indent groups
    return page_df.reset_index(), pruned_words_df, indent_groups


# Finding indentations associated with genus, epithet, infra

In [25]:
types = ['genus', 'epithet', 'infra', 'author', 'misc.']
def n_leftmost_indent(df, n):
    """return a tuple with at most 3 elements each element itself is a tuple containing indent group, mean, group len"""
    indent_groups = [(g, df[(df['indent_group'] == g) & (df['word_no'] == 0)]['x0'].mean(), len(df[(df['indent_group'] == g) & (df['word_no'] == 0)]['x0'])) for g in df['indent_group'].unique()]
    indent_groups.sort(key = lambda x : x[1])
    #print(indent_groups[:n])
    return indent_groups[:n]

In [26]:
def get_genusEpithetInfra_indent(col_df):
    leftmost_3_indents = n_leftmost_indent(col_df, 3) #for vol1 only 2 indentations will be given 
    min_gap = 0
    max_gap = 75 #error is 30 -- less than 50% of max gap (which will be ignored for now)

    # possibly not specific enough
    # first identifying indent based don distance from one another only
    """if len(leftmost_3_indents) == 3:
        if leftmost_3_indents[0][1] < max_gap:
            leftmost_3_indents = leftmost_3_indents[1:]
        elif ((leftmost_3_indents[1][1] - leftmost_3_indents[0][1]) > max_gap or \
            (leftmost_3_indents[1][1] - leftmost_3_indents[0][1]) < min_gap): #comparing first two (if satisfied last two will be checked in next if block)
            leftmost_3_indents = [max(leftmost_3_indents[1:], key = lambda x : x[2])] + [leftmost_3_indents[2]]
        elif (leftmost_3_indents[2][1] - leftmost_3_indents[1][1]) > max_gap or \
            (leftmost_3_indents[2][1] - leftmost_3_indents[1][1]) < min_gap: #comparing last two
            leftmost_3_indents = [leftmost_3_indents[0]] + [max(leftmost_3_indents[1:], key = lambda x : x[2])]

    if len(leftmost_3_indents) == 2:
        if leftmost_3_indents[0][1] < max_gap:
            leftmost_3_indents = leftmost_3_indents[1]
        elif (leftmost_3_indents[1][1] - leftmost_3_indents[0][1]) > max_gap or (leftmost_3_indents[1][1] - leftmost_3_indents[0][1]) < min_gap:
            leftmost_3_indents = [max(leftmost_3_indents, key = lambda x : x[2])]"""

    has_genus, has_epithet, has_infra = False, False, False
    genus_indent, epithet_indent, infra_indent = -1, -1, -1
    if len(leftmost_3_indents) == 3 and type(leftmost_3_indents) == type([1,2,3]):
        has_genus, has_epithet, has_infra = True, True, True
        #print("leftmost 3:", leftmost_3_indents)
        genus_indent, epithet_indent, infra_indent = [el[0] for el in leftmost_3_indents]
    elif len(leftmost_3_indents) == 2:
        if col_df[col_df['indent_group'] == leftmost_3_indents[1][0]]['word'].apply(is_infra).any():
            has_genus, has_epithet, has_infra = False, True, True
            epithet_indent, infra_indent = [el[0] for el in leftmost_3_indents]
        else:
            has_genus, has_epithet, has_infra = True, True, False
            genus_indent, epithet_indent = [el[0] for el in leftmost_3_indents]
    elif len(leftmost_3_indents) == 1 or type(leftmost_3_indents) == type((1,2,3)): 
        if type(leftmost_3_indents) == type((1,2,3)):
            leftmost_3_indents = [leftmost_3_indents]
        has_genus, has_epithet, has_infra = False, True, False
        epithet_indent = leftmost_3_indents[0][0]

    return genus_indent, epithet_indent, infra_indent, leftmost_3_indents

# Processing column dataframes


In [27]:
def process_col(col_df, genus, epithet, draw_genus, draw_epithet, draw_infra = np.NaN):
    genus_indent, epithet_indent, infra_indent, indent_3_left = get_genusEpithetInfra_indent(col_df)
    #print(genus_indent, epithet_indent, infra_indent, indent_3_left)
    
    blocks = col_df['block_no'].unique()
    start_word_cond = -1 
    author = ''
    #draw_infra = np.NaN

    process_hybrid = False
    process_infra = False
    hybrid_pre_str = ''

    col_df = col_df.copy()
    for index, row in col_df.iterrows():
        b, l, w = row['block_no'], row['line_no'], row['word_no']
        word, indent_group = row['word'], row['indent_group']
        row_cond = (col_df['line_no'] == l) & (col_df['block_no'] == b) & (col_df['word_no'] == w) 
        
        if w == 0 or process_hybrid: 
            if is_hybrid(word):
                process_hybrid = True
                misc = word
                author = ''
                hybrid_pre_str = 'hybrid-'
            
            else:
                if process_hybrid: 
                    #col_df.loc[start_word_cond, 'misc.'] = f"hybrid ({misc})"
                    process_hybrid = False
                    hybrid_pre_str = 'hybrid-'
                    #print("hi")
                else: 
                    hybrid_pre_str = ''

                prev_start_cond = start_word_cond
                start_word_cond = row_cond
                process_hybrid = False
                process_infra = False
                
                if indent_group == genus_indent and not ''.join(e for e in word if e.isalpha()).isupper():
                    process_infra = False
                    genus = word
                    draw_genus = genus
                    epithet = ''
                    draw_epithet = ''
                    author = ''
                    misc = ''
                    infra = ''
                    col_df.loc[start_word_cond, 'genus'] = genus
                    col_df.loc[start_word_cond, 'taxon rank'] = hybrid_pre_str  + 'genus'
                    if not is_genus(word):
                        col_df.loc[row_cond, 'error_check'] = True
                    col_df.loc[row_cond, 'draw_genus'] = draw_genus
                    col_df.loc[row_cond, 'author'] = ''

                elif indent_group == epithet_indent and not ''.join(e for e in word if e.isalpha()).isupper():
                    process_infra = False
                    epithet = word
                    author = ''
                    col_df.loc[row_cond, 'genus'] = genus
                    col_df.loc[row_cond, 'epithet'] = epithet
                    col_df.loc[row_cond, 'taxon rank'] = hybrid_pre_str  + 'species'
                    if not is_epithet(word):
                        col_df.loc[row_cond, 'error_check'] = True
                    draw_epithet = str(genus) + '_' + str(epithet) +'_' + str(b) + '_' + str(l)
                    col_df.loc[row_cond, 'draw_genus'] = draw_genus
                    col_df.loc[row_cond, 'draw_epithet'] = draw_epithet
                    col_df.loc[row_cond, 'author'] = ''

                elif indent_group == infra_indent and is_infra:
                    #col_df.loc[row_cond, 'misc.'] = word
                    process_infra = True
                    misc = word
                    author = ''
                    if not (is_infra(word) or is_hybrid(word)):
                        col_df.loc[start_word_cond, 'error_check'] = True
                        #possibly should add to author of previous line here? 
                        #print("possible author?", word)
                    #print("hi")

        elif w == 1 and process_infra and (is_infra(misc) or len(''.join(e for e in misc if e.isalpha())) <2) and (misc != "et"):
            infra = word 
            start_word_cond = row_cond
            #print(updated)
            #print(b, l, w)
            #col_df.loc[row_cond, 'misc.'] = misc
            col_df.loc[start_word_cond, 'taxon rank'] = hybrid_pre_str + misc
            col_df.loc[start_word_cond, 'genus'] = genus
            col_df.loc[start_word_cond, 'epithet'] = epithet
            col_df.loc[start_word_cond, 'infra'] = infra
            col_df.loc[start_word_cond, 'author'] = ''
            draw_infra = str(infra) + '_'+str(b)+'_'+str(l)
            process_infra = False
            col_df.loc[start_word_cond, 'draw_genus'] = draw_genus
            col_df.loc[start_word_cond, 'draw_epithet'] = draw_epithet
            col_df.loc[start_word_cond, 'draw_infra'] = draw_infra
        
        elif process_infra:
            #print(misc, word)
            if misc != "":
                curr_author_part = misc + ' ' + word + ' '
                col_df.loc[start_word_cond, 'draw_author'] = 'author_'+str(b)+'_'+str(l)
            else:
                curr_author_part = word + ' '
            col_df.loc[prev_start_cond, 'author'] += curr_author_part
            col_df.loc[row_cond, 'draw_author'] = 'author_'+str(b)+'_'+str(l)
            col_df.loc[row_cond, 'draw_genus'] = draw_genus

        elif (type(genus) == type("STR") and genus != '') or (type(epithet) == type("STR") and epithet != ''):
            curr_author_part = word + ' '
            col_df.loc[start_word_cond, 'author'] += curr_author_part
            col_df.loc[row_cond, 'draw_author'] = 'author_'+str(b)+'_'+str(l)
            col_df.loc[row_cond, 'draw_genus'] = draw_genus

    #Last author
    """if author != '':
        col_df.loc[start_word_cond, 'author'] = author"""
                    

    return col_df, genus, epithet, draw_genus, draw_epithet


# Run PreProcessing

In [28]:
#preprocessing
genus = np.NaN
df_dict = {}
pruned_dict = {}

for page_num in tqdm(index):
    page_df, pruned_df, indent_group = preprocessing(page_num)
    df_dict[page_num] = page_df
    pruned_dict[page_num] = pruned_df

genus = np.NaN
epithet = np.NaN
draw_genus = np.NaN
draw_epithet = np.NaN
result_ims_valid_words = []
df_list = []

for page_num in tqdm(index):
    #page_num = index[-1]
    #process the pre-processed dfs
    page_df = df_dict[page_num]
    
    #for drawing
    pix_map = doc.get_page_pixmap(page_num,matrix=mat)
    image = Image.open(io.BytesIO(pix_map.tobytes()))
    draw = ImageDraw.Draw(image)
    
    plot_valid_words(page_df, draw, color = '#660044', w = 2)
    result_ims_valid_words.append(image)
    
    #break 
#result_ims_valid_words[0].save(OUTPUT_PATH + "preprocessed/" + 'valid_words' + TAIL_STR + '.pdf',save_all=True, append_images=result_ims[1:])

  0%|          | 0/22 [00:00<?, ?it/s]/opt/homebrew/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
/opt/homebrew/lib/python3.9/site-packages/numpy/core/fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape
/opt/homebrew/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths

# Final Results + SAVE

In [29]:
#Setting up files and directories for saving the results
SCRIPT_NAME = "vol2_index_synonyms"
SCRIPT_OUTPUT_PATH = "../output/index/new_code/" + SCRIPT_NAME + "/"
DATE_STR = datetime.now().strftime("%Y_%m_%d") 
TIME_STR = datetime.now().strftime("%H%M")
QUICK_FIX = False
TAIL_STR = ''

if QUICK_FIX:
    OUTPUT_PATH = SCRIPT_OUTPUT_PATH + DATE_STR + "/QuickFix/" 
    #TAIL_STR = '_' + DATE_STR + '_' + TIME_STR
else:
    OUTPUT_PATH = SCRIPT_OUTPUT_PATH + DATE_STR + "/" + TIME_STR + "/"

try:
    os.makedirs(OUTPUT_PATH)
except FileExistsError:
    # directory already exists
    pass

try:
    os.makedirs(OUTPUT_PATH + "preprocessed/")
except FileExistsError:
    # directory already exists
    pass

try:
    os.makedirs(OUTPUT_PATH + 'raw/')
except FileExistsError:
    # directory already exists
    pass

In [30]:
genus = np.NaN
epithet = np.NaN
draw_genus = np.NaN
draw_epithet = np.NaN
result_ims = []
df_list = []

for page_num in tqdm(index):
    #if page_num == index[-2]:
    #    break
    #page_num = index[-1]
    #process the pre-processed dfs
    page_df = df_dict[page_num]
    
    #for drawing
    pix_map = doc.get_page_pixmap(page_num,matrix=mat)
    image = Image.open(io.BytesIO(pix_map.tobytes()))
    draw = ImageDraw.Draw(image)

    #processing each column
    for c in page_df['col_no'].unique():
        col_df = page_df[page_df['col_no'] == c]
        col_df, genus, epithet, draw_genus, draw_epithet = process_col(col_df, genus, epithet, draw_genus, draw_epithet)
        df_list.append(col_df)

        #drawing boxes in each column
        plot_genus_blocks(col_df, draw)
        plot_epithet_blocks(col_df, draw)
        plot_author_blocks(col_df, draw)
        plot_infra_blocks(col_df, draw)

    result_ims.append(image)
    #break 

#TIME_STR = datetime.now().strftime("%Y_%m_%d-%I_%M_%p")
result_ims[0].save(OUTPUT_PATH + 'vol2_index_ROI.pdf',save_all=True, append_images=result_ims[1:])

pre_processed_df = pd.concat([df_dict[k] for k in df_dict], axis = 0)
result_ims_valid_words[0].save(OUTPUT_PATH + "preprocessed/" + 'valid_words' + TAIL_STR + '.pdf',save_all=True, append_images=result_ims_valid_words[1:])
pre_processed_df.to_html(OUTPUT_PATH + "preprocessed/" + 'vol2_preprocessed_index' + TAIL_STR + '.html')
pre_processed_df.to_csv(OUTPUT_PATH + "preprocessed/" + 'vol2_preprocessed_index' + TAIL_STR + '.csv')

df = pd.concat(df_list, axis = 0)
df.to_html(OUTPUT_PATH + 'raw/' + 'vol2_index' + TAIL_STR + '.html')
df.to_csv(OUTPUT_PATH + 'raw/' + 'vol2_index' + TAIL_STR + '.csv', index = False)

pruned = df[(~df['genus'].isnull())]
pruned = pruned[["page_num", "genus", "epithet", "infra" ,"author", "taxon rank"]]
pruned.to_csv(OUTPUT_PATH + 'vol2_index_pruned' + TAIL_STR + '.csv', index = False)
pruned.to_html(OUTPUT_PATH + 'vol2_index_pruned' + TAIL_STR + '.html')

100%|██████████| 22/22 [00:05<00:00,  3.70it/s]
